In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np

from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.feature_extraction import DataSource
from pals.PLAGE import PLAGE
from pals.ORA import ORA
from pals.GSEA import GSEA
from pals.common import *

from pals.preprocessing import *

from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests



2021-09-01 15:03:12.341 | WARNING  | pals.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2021-09-01 15:03:12.343 | WARNING  | pals.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2021-09-01 15:03:14.481 | INFO     | pals.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


In [3]:
mz_df = pd.DataFrame(columns = ['m/z', 'retention_time'])

mz_df['m/z'] = [np.random.randint(1, 150) for i in range(7375)]
mz_df['retention_time'] = [np.random.randint(1, 300) for i in range(7375)]
print(mz_df)


      m/z  retention_time
0      48             108
1      91             216
2      57             175
3      10             230
4     137             218
...   ...             ...
7370   98             214
7371   50             131
7372    1              36
7373  141             106
7374   40              52

[7375 rows x 2 columns]


In [4]:
intdf, annodf, groups = load_data("int_df.csv", "annotation_df.csv") #path to beer data 
groups

2021-09-01 15:03:15.811 | DEBUG    | pals.common:load_data:174 - Loaded 7375 x 12 peak intensities from int_df.csv
2021-09-01 15:03:15.811 | DEBUG    | pals.common:load_data:175 - Loaded groups: {'beer1': ['Beer_1_full1.mzXML', 'Beer_1_full2.mzXML', 'Beer_1_full3.mzXML'], 'beer2': ['Beer_2_full1.mzXML', 'Beer_2_full2.mzXML', 'Beer_2_full3.mzXML'], 'beer3': ['Beer_3_full1.mzXML', 'Beer_3_full2.mzXML', 'Beer_3_full3.mzXML'], 'beer4': ['Beer_4_full1.mzXML', 'Beer_4_full2.mzXML', 'Beer_4_full3.mzXML']}
2021-09-01 15:03:15.825 | DEBUG    | pals.common:load_data:178 - Loaded 14549 peak annotations from annotation_df.csv


{'beer1': ['Beer_1_full1.mzXML', 'Beer_1_full2.mzXML', 'Beer_1_full3.mzXML'],
 'beer2': ['Beer_2_full1.mzXML', 'Beer_2_full2.mzXML', 'Beer_2_full3.mzXML'],
 'beer3': ['Beer_3_full1.mzXML', 'Beer_3_full2.mzXML', 'Beer_3_full3.mzXML'],
 'beer4': ['Beer_4_full1.mzXML', 'Beer_4_full2.mzXML', 'Beer_4_full3.mzXML']}

In [5]:
comparisons = [
    ('beer1', 'beer2'), 
    ('beer3', 'beer4')
]

In [6]:
experimental_design = {
    'groups': groups,
    'comparisons': []
}
for case, control in comparisons:
    experimental_design['comparisons'].append({
        'case': case,
        'control': control,
        'name': '%s/%s' % (case, control)
    })
    

In [7]:
ds = DataSource(intdf, annodf, experimental_design, DATABASE_PIMP_KEGG)

2021-09-01 15:03:15.877 | DEBUG    | pals.feature_extraction:__init__:44 - Using PiMP_KEGG as database
2021-09-01 15:03:15.878 | DEBUG    | pals.loader:load_data:42 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2021-09-01 15:03:15.913 | DEBUG    | pals.feature_extraction:__init__:57 - Mapping pathway to unique ids
2021-09-01 15:03:15.920 | DEBUG    | pals.feature_extraction:__init__:71 - Creating dataset to pathway mapping
2021-09-01 15:03:16.443 | DEBUG    | pals.feature_extraction:__init__:99 - Computing unique id counts


In [8]:
#Method for calculating p values from intensity matrix

def calculate_p(intensity_df):
    #students t for generating p values per compound

    comp = experimental_design['comparisons'][0]

    control_groups = ds.get_comparison_samples(comp)[0]
    case_groups = ds.get_comparison_samples(comp)[1]
    
    df = pd.DataFrame(columns = ['case', 'control']) #create case control df

    for sample_group in control_groups:
        df['control'] = intdf[sample_group] #populate with control data

    for sample_group in case_groups:
        df['case'] = intdf[sample_group] #populate with case data

    pvals = []

    df['pvalue'] = 0 #initialise p value column to 0
    df['control'] = np.random.normal(size=len(df.index)) + 4
    df['case'] = np.random.normal(size=len(df.index)) + 4
    
    
    for i in range(len(df.index)):
        if str(df['case'].iloc[i]) == 'nan':
            df['case'].iloc[i] == 5000
        if str(df['control'].iloc[i]) == 'nan':
            df['control'].iloc[i] == 5000
    
    print(df)
    
    #df.to_csv("tst.csv")


    for i in range(len(df.index)): #iterate through rows in dataframe calculating p value for each comparison
        
        case_log = np.log2(df['case'].iloc[i])
        control_log = np.log2(df['control'].iloc[i])

        if str(case_log) == 'nan': #filter nan values
            case_log = 1
        if str(control_log) == 'nan':
            control_log = 1

        statistics, pvalue = stats.ttest_ind(df['case'].iloc[i], df['control'].iloc[i]) #do t test on the logged 2 values
        pvals.append(pvalue) # append to pval list 


#     print(pvals) #debug list

In [9]:
calculate_p(intdf)

c:\users\joewa\.virtualenvs\pals-4ls44jcw\lib\site-packages\numpy\core\fromnumeric.py:3724: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
c:\users\joewa\.virtualenvs\pals-4ls44jcw\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


             case   control  pvalue
row_id                             
3033929  4.452105  2.929369       0
3033930  3.948992  4.111505       0
3033931  6.040421  4.360116       0
3033932  4.297623  4.103223       0
3033933  4.093104  3.317280       0
...           ...       ...     ...
3041299  3.795027  2.707230       0
3041300  5.154281  4.467177       0
3041301  5.508620  6.838781       0
3041302  3.627212  4.150523       0
3041303  5.088365  4.320871       0

[7375 rows x 3 columns]


### Joe checking from this point onwards ...

With commentaries ...

I'll copy and paste some lines of codes from the `calculate_p` method above, but breaking it into smaller cells, so it's easier to inspect the results interactively.
In general, this is how I (and most people) tend to use the notebook environment. Once the codes runs then we put it togethr into a separate .py files for use elsewhere.

In [10]:
intensity_df = intdf

In [11]:
comp = experimental_design['comparisons'][0]
control_groups = ds.get_comparison_samples(comp)[0]
case_groups = ds.get_comparison_samples(comp)[1]
control_groups, case_groups

(['Beer_2_full1.mzXML', 'Beer_2_full2.mzXML', 'Beer_2_full3.mzXML'],
 ['Beer_1_full1.mzXML', 'Beer_1_full2.mzXML', 'Beer_1_full3.mzXML'])

In [12]:
df = pd.DataFrame(columns = ['case', 'control']) #create case control df

for sample_group in control_groups:
    df['control'] = intdf[sample_group] #populate with control data

for sample_group in case_groups:
    df['case'] = intdf[sample_group] #populate with case data

df

,case,control
row_id,,
3033929,2.170697e+09,1.959480e+09
3033930,4.894853e+07,3.908452e+07
3033931,1.585143e+09,1.555666e+09
3033932,5.914975e+08,4.038747e+08
3033933,1.092635e+09,1.192834e+09
...,...,...
3041299,1.478325e+04,1.284756e+04
3041300,2.756565e+04,2.517871e+04
3041301,2.165889e+04,2.039137e+04


So ... above dataframe doesn't look correct as we only see one column for case and control. That's because in the for-loop below: 
```
    df['control'] = intdf[sample_group] #populate with control data
```
We loop over the sample name (`sample_group`), and assign the values of each sample to the same column (`control`) repeatly, so only the last column is kept in the dataframe.
You can do t-test with only one sample in each group.

I believe what you want is this:

First, get the entire measurement dataframe from `DataSource`

In [13]:
full_df = ds.get_measurements()
full_df

,Beer_1_full1.mzXML,Beer_1_full2.mzXML,Beer_1_full3.mzXML,Beer_2_full1.mzXML,Beer_2_full2.mzXML,Beer_2_full3.mzXML,Beer_3_full1.mzXML,Beer_3_full2.mzXML,Beer_3_full3.mzXML,Beer_4_full1.mzXML,Beer_4_full2.mzXML,Beer_4_full3.mzXML
row_id,,,,,,,,,,,,
3033929,2.235291e+09,2.000478e+09,2.170697e+09,2.242760e+09,2.279882e+09,1.959480e+09,2.079356e+09,2.110473e+09,2.243653e+09,1.817065e+09,1.746443e+09,1.779827e+09
3033930,4.433491e+07,4.287387e+07,4.894853e+07,4.760448e+07,4.217280e+07,3.908452e+07,3.825778e+07,3.770192e+07,4.087189e+07,3.330477e+07,3.153630e+07,3.102410e+07
3033931,1.723985e+09,1.764235e+09,1.585143e+09,1.543961e+09,1.579320e+09,1.555666e+09,1.698130e+09,1.481824e+09,1.508645e+09,1.642510e+09,1.723919e+09,1.697806e+09
3033932,6.254237e+08,6.503417e+08,5.914975e+08,4.635929e+08,4.298382e+08,4.038747e+08,4.292837e+08,3.708761e+08,4.778932e+08,3.903165e+08,4.080995e+08,4.309892e+08
3033933,1.075022e+09,9.293474e+08,1.092635e+09,1.130720e+09,1.118146e+09,1.192834e+09,1.231442e+09,1.262046e+09,1.460653e+09,1.009838e+09,9.085111e+08,9.967176e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
3041299,1.431211e+04,6.565678e+03,1.478325e+04,1.620252e+04,1.748920e+04,1.284756e+04,3.306687e+04,2.476216e+04,2.869417e+04,2.231166e+04,2.164017e+04,2.727751e+04
3041300,2.273721e+04,2.905976e+04,2.756565e+04,3.080164e+04,2.427240e+04,2.517871e+04,2.718543e+04,2.905361e+04,3.170420e+04,2.597168e+04,3.066904e+04,2.884984e+04
3041301,1.760107e+04,2.674373e+04,2.165889e+04,2.121242e+04,1.737357e+04,2.039137e+04,2.296079e+04,2.001743e+04,2.664124e+04,1.791359e+04,1.642459e+04,2.349759e+04


Then select the columns that we want

In [14]:
selected = case_groups + control_groups
selected_df = full_df[selected]
selected_df

,Beer_1_full1.mzXML,Beer_1_full2.mzXML,Beer_1_full3.mzXML,Beer_2_full1.mzXML,Beer_2_full2.mzXML,Beer_2_full3.mzXML
row_id,,,,,,
3033929,2.235291e+09,2.000478e+09,2.170697e+09,2.242760e+09,2.279882e+09,1.959480e+09
3033930,4.433491e+07,4.287387e+07,4.894853e+07,4.760448e+07,4.217280e+07,3.908452e+07
3033931,1.723985e+09,1.764235e+09,1.585143e+09,1.543961e+09,1.579320e+09,1.555666e+09
3033932,6.254237e+08,6.503417e+08,5.914975e+08,4.635929e+08,4.298382e+08,4.038747e+08
3033933,1.075022e+09,9.293474e+08,1.092635e+09,1.130720e+09,1.118146e+09,1.192834e+09
...,...,...,...,...,...,...
3041299,1.431211e+04,6.565678e+03,1.478325e+04,1.620252e+04,1.748920e+04,1.284756e+04
3041300,2.273721e+04,2.905976e+04,2.756565e+04,3.080164e+04,2.427240e+04,2.517871e+04
3041301,1.760107e+04,2.674373e+04,2.165889e+04,2.121242e+04,1.737357e+04,2.039137e+04


Now we can do t-test

In [15]:
results = []
for idx, row in selected_df.iterrows(): #iterate through rows in dataframe calculating p value for each comparison
    case_log = np.log2(row[case_groups].values)
    control_log = np.log2(row[control_groups].values)        
    statistics, pvalue = stats.ttest_ind(case_log, control_log)
    lfc = np.mean(case_log) - np.mean(control_log)
    res = [idx, pvalue, lfc]
    results.append(res)

In [16]:
result_df = pd.DataFrame(results, columns=['idx', 'pvalue', 'lfc'])
result_df

,idx,pvalue,lfc
0,3033929,0.865085,-0.015246
1,3033930,0.461829,0.081933
2,3033931,0.073748,0.115310
3,3033932,0.001662,0.526615
4,3033933,0.122688,-0.155425
...,...,...,...
7370,3041299,0.316927,-0.463320
7371,3041300,0.921725,-0.015859
7372,3041301,0.494204,0.146687
7373,3041302,0.155722,0.346735


Don't forget to correct for multiple tests here ...

In [17]:
# add your codes

PS. you should also consider the fact that users can specify multiple comparisons, e.g.

In [18]:
ds.get_experimental_design()['comparisons']

[{'case': 'beer1', 'control': 'beer2', 'name': 'beer1/beer2'},
 {'case': 'beer3', 'control': 'beer4', 'name': 'beer3/beer4'}]

In this case, we should run the t-test (and mummichog?) multiple times, one for each comparison. Take a look at the output of ORA for this data source. We see two sets of p-values for the pathways, one for beer1/beer2, another for beer3/beer4. It would be really nice if the mummichog codes can support this too for compatibility with other methods in PALS.

In [19]:
from pals.ORA import ORA

ora = ORA(ds)
pathway_df = ora.get_results()

2021-09-01 15:03:24.354 | DEBUG    | pals.ORA:__init__:21 - ORA initialised
2021-09-01 15:03:24.466 | DEBUG    | pals.ORA:get_results:47 - Calculating ORA
2021-09-01 15:03:24.468 | DEBUG    | pals.preprocessing:process:17 - Replacing negative and zero values with NaN
2021-09-01 15:03:24.473 | DEBUG    | pals.preprocessing:process:29 - Performing min-value imputation
2021-09-01 15:03:24.479 | DEBUG    | pals.preprocessing:process:42 - Performing row average imputation
2021-09-01 15:03:24.493 | DEBUG    | pals.preprocessing:process:70 - Applying log normalisation
2021-09-01 15:03:29.217 | DEBUG    | pals.ORA:get_results:104 - Correcting for multiple t-tests
2021-09-01 15:03:29.220 | DEBUG    | pals.feature_extraction:_calculate_coverage_df:362 - Calculating dataset formula coverage


In [20]:
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,PiMP_KEGG beer1/beer2 comb_p,PiMP_KEGG beer3/beer4 comb_p,unq_pw_F,tot_ds_F,F_coverage
map00232,Caffeine metabolism,5.058474e-01,5.402563e-01,7.235539e-01,7.404068e-01,15,4,26.67
map00940,Phenylpropanoid biosynthesis,2.551922e-06,1.412937e-06,3.204079e-05,1.878376e-05,50,25,50.00
map00600,Sphingolipid metabolism,5.466354e-01,2.744439e-01,7.487249e-01,4.845650e-01,10,3,30.00
map00720,Carbon fixation pathways in prokaryotes,1.886792e-01,2.287481e-01,3.773584e-01,4.266524e-01,37,9,24.32
map00260,"Glycine, serine and threonine metabolism",3.791629e-06,8.033401e-06,4.408709e-05,8.252494e-05,41,21,51.22
...,...,...,...,...,...,...,...,...
map00052,Galactose metabolism,5.693949e-09,1.159169e-08,1.838332e-07,2.619722e-07,21,17,80.95
map00230,Purine metabolism,2.862426e-01,4.711185e-01,4.938231e-01,7.004789e-01,78,21,26.92
map00626,Naphthalene degradation,1.202364e-02,2.676564e-03,4.605666e-02,1.287029e-02,43,17,39.53
map00450,Selenocompound metabolism,9.829573e-01,9.862673e-01,1.000000e+00,1.000000e+00,21,1,4.76


Finally as mentioned before, there's a number of preprocessing classes you can use in your method if you want: https://github.com/glasgowcompbio/PALS/blob/master/pals/feature_extraction.py.

Example usage: 
- https://github.com/glasgowcompbio/PALS/blob/master/pals/ORA.py#L26-L35
- https://github.com/glasgowcompbio/PALS/blob/master/pals/PLAGE.py#L43-L53